# **Local-DreamBooth colab From https://github.com/TheLastBen/fast-stable-diffusion, if you face any issues, feel free to discuss them.** 
Keep your notebook updated for best experience.

This is a windows version, no support for bitsandbytes or DeepSpeed, it will require more than 18GB of VRAM. A Linux version is in progress ...
To connect a local runtime, run these commands in Anaconda :

In [ ]:
#@markdown #Dependencies, run only once, make sure you have all A1111 dependencies installed before running this cell, including xformers.

#1- pip install jupyter_http_over_ws
#2- jupyter serverextension enable --py jupyter_http_over_ws
#3- jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com'   --port=8888  --NotebookApp.port_retries=0 --no-browser
#4- Use the link given : "http://localhost:8888/?token=xxxxxx" as the local server in google colab

from subprocess import getoutput
import os

!git clone --depth 1 https://github.com/TheLastBen/diffusers
!pip uninstall -q -y diffusers
!pip uninstall -q -y transformers
!pip install -q git+https://github.com/TheLastBen/diffusers
!pip uninstall -y -q accelerate
!pip install -q accelerate==0.12.0
!pip install -q OmegaConf
!pip install -q -U wandb
!pip uninstall -y -q wget
!pip install -q wget==3.2
!pip install py7zr
import wget
wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/Deps')
!ren Deps Deps.7z
!py7zr x Deps.7z
s=getoutput('pip show wget')
s=s[213:259]
!xcopy /E /Q /H /Y "$MAIN_DIR/usr/local/lib/python3.7/dist-packages" "$s"
os.remove('Deps.7z')
!rd /S /Q "usr"
print('[1;31mDone!')

# Dreambooth

In [ ]:
import os
from IPython.display import clear_output
from IPython.utils import capture
import wget
import time

#@markdown #Create/Load Session

#@markdown ---

try:
   if Session_Name:
     pass
except:   
   MAIN_DIR=os.getcwd()

Session_Name = "" #@param{type: 'string'}
while Session_Name=="":
  print('[1;31mInput the Session Name:')
  Session_Name=input('')
INSTANCE_NAME=Session_Name


#@markdown - To resume a previous session, just enter its name, it if it exists, it will load it, otherwise a new session will be created.

WORKSPACE=MAIN_DIR+'/Fast-Dreambooth'
MODEL_NAME=MAIN_DIR+'/stable-diffusion-v1-5'
OUTPUT_DIR=MAIN_DIR+'/models/'+Session_Name
SESSION_DIR=WORKSPACE+'/Sessions/'+Session_Name
INSTANCE_DIR=WORKSPACE+'/Sessions/'+Session_Name+'/instance_images'
MDLPTH=str(SESSION_DIR+'/'+Session_Name+'.ckpt')

if os.path.exists(str(SESSION_DIR)) and not os.path.exists(str(SESSION_DIR+"/"+Session_Name+'.ckpt')):
  print('[1;32mSession found, Loading session....')
  while not os.path.exists(MODEL_NAME+'/unet/diffusion_pytorch_model.bin'):
      print('[1;31mNo model found, make sure you put the diffusers model in the right folder')
      time.sleep(5)
  print('[1;32mSession loaded with no previous CKPT.')

elif os.path.exists(str(SESSION_DIR+"/"+Session_Name+'.ckpt')):
  print('[1;32mSession found, loading the model, this might take a few minutes...')
  if not os.path.exists(str(OUTPUT_DIR)):
    %mkdir "$OUTPUT_DIR"
  with capture.capture_output() as cap:
    %cd $MAIN_DIR
    wget.download('https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/convertodiffloc.py', MAIN_DIR)
  !python convertodiffloc.py --checkpoint_path "$MDLPTH" --dump_path "$OUTPUT_DIR"
  if os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
    resume=True
    os.remove('convertodiffloc.py')
    os.remove('v1-inference.yaml')
    clear_output()
    print('[1;32mSession loaded with the trained model')
  else:
    os.remove('convertodiffloc.py')
    os.remove('v1-inference.yaml')
    while not os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
      print('[1;31mConversion error, it appears the the CKPT from this session is incompatible or corrupt, remove it to continue')
      time.sleep(5)


elif not os.path.exists(str(SESSION_DIR)):
    %mkdir "$INSTANCE_DIR"
    print('[1;32mCreating session...')
    while not os.path.exists(MODEL_NAME+'/unet/diffusion_pytorch_model.bin'):
        print('[1;31mNo model found, make sure you put the diffusers model in the right folder')
        time.sleep(5)
    print('[1;32mSession Created.')
    
#@markdown 

#@markdown # The most importent step is to rename the instance picture to the same instance unique identifier for each subject, example :
#@markdown - If you have 30 pictures of yourself, simply select them all and rename only one to the chosen identifier for example : phtmejhn, the files would be : phtmejhn (1).jpg, phtmejhn (2).png ....etc then upload them, do the same for other people or objects with a different identifier, and that's it.
#@markdown - Check out this example : https://i.imgur.com/d2lD3rz.jpeg
      


In [ ]:
#@markdown #Instance Images
#@markdown ----

#@markdown
#@markdown - Run the cell to Upload the instance pictures.



Remove_existing_instance_images= True #@param{type: 'boolean'}
#@markdown - This will remove the previous instance images, uncheck to add the new isntance pictures to the existing ones (if any).


if Remove_existing_instance_images:
  if os.path.exists(str(INSTANCE_DIR)):
    !rd /S /Q "$INSTANCE_DIR"

if not os.path.exists(str(INSTANCE_DIR)):
  %mkdir "$INSTANCE_DIR"

IMAGES__FOLDER="" #@param{type: 'string'}
IMAGES_FOLDER=MAIN_DIR+'/'+IMAGES__FOLDER

#@markdown - Enter the path of the folder containing your instance images

while IMAGES__FOLDER !="" and not os.path.exists(str(IMAGES_FOLDER)):
  print('[1;31mThe image folder specified does not exist, use the colab file explorer to copy the path')
  time.sleep(5)


if IMAGES__FOLDER!="":
  with capture.capture_output() as cap:
    !xcopy /Y "$IMAGES_FOLDER" "$INSTANCE_DIR" 
    if os.path.exists(INSTANCE_DIR+"/.ipynb_checkpoints"):
      !rd /S /Q  INSTANCE_DIR+"/.ipynb_checkpoints"      
  print('[1;32mDone, proceed to the training cell')
elif IMAGES__FOLDER=="":
  print(('[1;31mProceeding without uploading instance images.'))

%cd "$INSTANCE_DIR"
[os.rename(f, f.replace(' ', '_')) for f in os.listdir('.') if not f.startswith('.')]
%cd $MAIN_DIR

# Training

In [ ]:
#@markdown ---
#@markdown #Start DreamBooth
#@markdown ---

Resume_Training = False #@param {type:"boolean"}

while not Resume_Training and not os.path.exists(MODEL_NAME+'/unet/diffusion_pytorch_model.bin'):
   print('[1;31mNo model found, make sure you put the diffusers model in the right folder')
   time.sleep(5)

#@markdown  - If you're not satisfied with the result, check this box, run again the cell and it will continue training the current model.

MODELT_NAME=MODEL_NAME

Training_Steps=3000 #@param{type: 'number'}
#@markdown - Total Steps = Number of Instance images * 100, if you use 30 images, use 3000 steps, if you're not satisfied with the result, resume training for another 500 steps, and so on ...

Seed=96576 #@param{type: 'number'}


try:
   resume
   if resume and not Resume_Training:
     print('[1;31mOverwrite your previously trained model ?, answering "yes" will train a new model, answering "no" will resume the training of the previous model?  yes or no ?[0m')
     while True:
        ansres=input('')
        if ansres=='no':
          Resume_Training = True
          del ansres
          break
        elif ansres=='yes':
          Resume_Training = False
          resume= False
          break
except:
  pass

if Resume_Training and os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
  MODELT_NAME=OUTPUT_DIR
  print('[1;32mResuming Training...[0m')
elif Resume_Training and not os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
  MODELT_NAME=MODEL_NAME  
  print('[1;31mPrevious model not found, training a new model...[0m') 



!accelerate launch $MAIN_DIR/diffusers/examples/dreambooth/train_dreambooth.py \
  --image_captions_filename \
  --train_text_encoder \
  --pretrained_model_name_or_path="$MODELT_NAME" \
  --instance_data_dir="$INSTANCE_DIR" \
  --output_dir="$OUTPUT_DIR" \
  --instance_prompt="" \
  --seed=$Seed \
  --resolution=512 \
  --mixed_precision="fp16" \
  --sample_batch_size=1 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --learning_rate=2e-6 \
  --lr_scheduler="polynomial" \
  --center_crop \
  --lr_warmup_steps=0 \
  --max_train_steps=$Training_Steps

if os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
  print("Almost done ...")
  %cd "$
  !wget -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
  clear_output()
  !sed -i '201s@.*@    model_path = "{OUTPUT_DIR}"@' $MAIN_DIR/convertosd.py
  !sed -i '202s@.*@    checkpoint_path= "{SESSION_DIR}/{Session_Name}.ckpt"@' $MAIN_DIR/convertosd.py
  !python $MAIN_DIR/convertosd.py
  clear_output()
  if os.path.exists(SESSION_DIR+"/"+INSTANCE_NAME+'.ckpt'):
    print("[1;32mDONE, the CKPT model is in the sessions folder")
  else:
    print("[1;31mSomething went wrong")
    
else:
  print("[1;31mSomething went wrong")